In [1]:
import tensorflow as tf

import pandas as pd
import numpy as np

import tensorflow as tf
from tensorflow.keras import optimizers
from tensorflow.keras.models import Model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import RepeatVector
from tensorflow.keras.layers import TimeDistributed
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Bidirectional

from tensorflow.python.keras import backend as K

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

import matplotlib.pyplot as plt
import seaborn as sns

from numpy.random import seed


SEED = 123  # used to help randomly select the data points
DATA_SPLIT_PCT = 0.2

df = pd.read_csv("reanalysis.csv")

2024-05-21 17:14:03.704433: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-05-21 17:14:03.785017: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-05-21 17:14:04.221781: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-21 17:14:06.365521: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
# SELECT THE TARGET

TARGET = "tas"

df=df.rename(columns={TARGET:'y'})

#Delete all the columns related to a daily scale
for col in df.columns:
    if "_" not in col and col != "y" and col != "time":
        df.drop(col, axis=1, inplace=True)

df["month"] = pd.to_datetime(df["time"]).dt.month
df["hour"] = pd.to_datetime(df["time"]).dt.hour

df.head()

,time,tas_daily,tas_min_daily,tas_max_daily,y,sfcWind_daily,pr_daily,sp_daily,month,hour
0,2000-01-01 00:00:00,295.3574,289.21075,301.2505,295.12326,4.347946,0.018645,100092.25,1,0
1,2000-01-01 01:00:00,295.3574,289.21075,301.2505,294.91876,4.347946,0.018645,100092.25,1,1
2,2000-01-01 02:00:00,295.3574,289.21075,301.2505,294.25897,4.347946,0.018645,100092.25,1,2
3,2000-01-01 03:00:00,295.3574,289.21075,301.2505,291.07550,4.347946,0.018645,100092.25,1,3
4,2000-01-01 04:00:00,295.3574,289.21075,301.2505,290.78757,4.347946,0.018645,100092.25,1,4


In [3]:
# For the target variable get the daily previous value and the daily next value
df["date"] = df["time"].apply(lambda x: x.split(" ")[0])
df_2 = df.copy()
df_2 = df_2[["date", TARGET + "_daily"]].groupby("date").mean()
df_2[TARGET+"_prev_daily"] = df_2[TARGET + "_daily"].shift(1)
df_2[TARGET+"_next_daily"] = df_2[TARGET + "_daily"].shift(-1)
df_2.drop(TARGET + "_daily", axis=1, inplace=True)
df_2.dropna(inplace=True)
df = df.merge(df_2, on="date", how="inner").drop("date", axis=1)

In [4]:
# Convert Categorical column to hot dummy columns
hotencoding1 = pd.get_dummies(df['month'])
hotencoding1 = hotencoding1.add_prefix('month')
hotencoding2 = pd.get_dummies(df['hour'])
hotencoding2 = hotencoding2.add_prefix('hour')

df=df.drop(['month', 'hour'], axis=1)

df=pd.concat([df, hotencoding1, hotencoding2], axis=1)

#Filter the rows with time less than 2015-01-01
df_train = df[df['time'] < '2013-12-31 23:59:00']
df_test = df[df['time'] >= '2013-12-31 23:59:00']

df_train = df_train.set_index('time')
df_test = df_test.set_index('time')

In [5]:
input_X_train = df_train.loc[:, df_train.columns != 'y'].values  # converts df to numpy array
input_X_test = df_test.loc[:, df_test.columns != 'y'].values  
input_y_train = df_train['y'].values
input_y_test = df_test['y'].values

n_features = input_X_train.shape[1]  # number of features

In [6]:
def temporalize(X, y, lookback):
    '''
    Inputs
    X         A 2D numpy array ordered by time of shape: (n_observations x n_features)
    y         A 1D numpy array with indexes aligned with X, i.e. y[i] should correspond to X[i]. Shape: n_observations.
    lookback  The window size to look back in the past records. Shape: a scalar.

    Output
    output_X  A 3D numpy array of shape: ((n_observations-lookback-1) x lookback x n_features)
    output_y  A 1D array of shape: (n_observations-lookback-1), aligned with X.
    '''
    output_X = []
    output_y = []
    for i in range(len(X) - lookback - 1):
        t = []
        for j in range(1, lookback + 1):
            # Gather the past records upto the lookback period
            t.append(X[[(i + j + 1)], :])
        output_X.append(t)
        output_y.append(y[i + lookback + 1])
    return np.squeeze(np.array(output_X)), np.array(output_y)

def flatten(X):
    '''
    Flatten a 3D array.

    Input
    X            A 3D array for lstm, where the array is sample x timesteps x features.

    Output
    flattened_X  A 2D array, sample x features.
    '''
    flattened_X = np.empty(
        (X.shape[0], X.shape[2]))  # sample x features array.
    for i in range(X.shape[0]):
        flattened_X[i] = X[i, (X.shape[1] - 1), :]
    return flattened_X

def scale(X, scaler):
    '''
    Scale 3D array.

    Inputs
    X            A 3D array for lstm, where the array is sample x timesteps x features.
    scaler       A scaler object, e.g., sklearn.preprocessing.StandardScaler, sklearn.preprocessing.normalize

    Output
    X            Scaled 3D array.
    '''
    for i in range(X.shape[0]):
        X[i, :, :] = scaler.transform(X[i, :, :])

    return X

In [7]:
lookback = 5
X_train, y_train = temporalize(X=input_X_train, 
                      y=input_y_train, 
                      lookback=lookback)
X_test, y_test = temporalize(X=input_X_test, 
                      y=input_y_test, 
                      lookback=lookback)
X_train, X_valid, y_train, y_valid = train_test_split(
    X_train,
    y_train,
    test_size=DATA_SPLIT_PCT,
    random_state=SEED)

TIMESTEPS = X_train.shape[1]  # equal to the lookback
N_FEATURES = X_train.shape[2]  # the number of features

In [8]:
# Initialize a scaler using the training data.
scaler = StandardScaler().fit(flatten(X_train))
X_train_scaled = scale(X_train, scaler)
X_valid_scaled = scale(X_valid, scaler)
X_test_scaled =  scale(X_test, scaler)

# Models

In [9]:
# To save the models with their respective rmse
models = []
rmse = []

In [10]:
lstm_baseline = Sequential()
lstm_baseline.add(Input(shape=(TIMESTEPS, N_FEATURES), 
                name='input'))
lstm_baseline.add(
    LSTM(units=16, 
         activation='tanh',
         recurrent_activation='sigmoid',
         return_sequences=True, 
         name='lstm_layer_1'))
lstm_baseline.add(
    LSTM(units=8, 
         activation='tanh', 
         recurrent_activation='sigmoid',
         return_sequences=False, 
         name='lstm_layer_2'))
lstm_baseline.add(Dense(units=1, 
                activation='linear', 
                name='output'))
lstm_baseline.compile(optimizer='adam',
              loss='mse',
              metrics=[
                  tf.keras.metrics.RootMeanSquaredError()
              ])
lstm_baseline_history = lstm_baseline.fit(x= np.asarray(X_train_scaled).astype('float32'),
                y=y_train,
                batch_size=128,
                epochs=100,
                validation_data=(np.asarray(X_valid_scaled).astype('float32'), 
                                y_valid),
                verbose=1).history

models.append(lstm_baseline)
rmse.append(lstm_baseline_history["val_root_mean_squared_error"][-1])

Epoch 1/100


2024-05-21 17:14:34.191633: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 86317440 exceeds 10% of free system memory.


767/767 ━━━━━━━━━━━━━━━━━━━━ 10s 8ms/step - loss: 81848.9609 - root_mean_squared_error: 286.0840 - val_loss: 77820.1094 - val_root_mean_squared_error: 278.9626
Epoch 2/100
767/767 ━━━━━━━━━━━━━━━━━━━━ 5s 7ms/step - loss: 76806.3906 - root_mean_squared_error: 277.1379 - val_loss: 73978.3594 - val_root_mean_squared_error: 271.9896
Epoch 3/100
767/767 ━━━━━━━━━━━━━━━━━━━━ 5s 7ms/step - loss: 73028.8281 - root_mean_squared_error: 270.2366 - val_loss: 70317.3750 - val_root_mean_squared_error: 265.1742
Epoch 4/100
767/767 ━━━━━━━━━━━━━━━━━━━━ 5s 7ms/step - loss: 69379.1406 - root_mean_squared_error: 263.3976 - val_loss: 66784.5312 - val_root_mean_squared_error: 258.4270
Epoch 5/100
767/767 ━━━━━━━━━━━━━━━━━━━━ 6s 7ms/step - loss: 65887.8750 - root_mean_squared_error: 256.6845 - val_loss: 63361.8203 - val_root_mean_squared_error: 251.7177
Epoch 6/100
767/767 ━━━━━━━━━━━━━━━━━━━━ 6s 7ms/step - loss: 62510.9453 - root_mean_squared_error: 250.0199 - val_loss: 60040.6875 - val_root_mean_squared_e

In [11]:
lstm_unrestricted = Sequential()
lstm_unrestricted.add(Input(shape=(TIMESTEPS, N_FEATURES),
                name='input'))
lstm_unrestricted.add(
    LSTM(units=16,
         activation='tanh',
         recurrent_activation='sigmoid',
         return_sequences=True,
         name='lstm_layer_1'))
lstm_unrestricted.add(
    LSTM(units=8,
         activation='tanh',
         recurrent_activation='sigmoid',
         return_sequences=True, 
         name='lstm_layer_2'))
lstm_unrestricted.add(Flatten())
lstm_unrestricted.add(Dense(units=1,
                activation='sigmoid', 
                name='output'))

lstm_unrestricted.compile(optimizer='adam',
              loss='mse',
              metrics=[
                  tf.keras.metrics.RootMeanSquaredError()
              ])
lstm_unrestricted_history = lstm_unrestricted.fit(x= np.asarray(X_train_scaled).astype('float32'),
                y=y_train,
                batch_size=128,
                epochs=100,
                validation_data=(np.asarray(X_valid_scaled).astype('float32'), 
                                y_valid),
                verbose=1).history

models.append(lstm_unrestricted)
rmse.append(lstm_unrestricted_history["val_root_mean_squared_error"][-1])

Epoch 1/100


2024-05-21 17:26:53.462279: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 86317440 exceeds 10% of free system memory.


767/767 ━━━━━━━━━━━━━━━━━━━━ 29s 23ms/step - loss: 84132.6406 - root_mean_squared_error: 290.0562 - val_loss: 84090.6172 - val_root_mean_squared_error: 289.9838
Epoch 2/100
767/767 ━━━━━━━━━━━━━━━━━━━━ 14s 18ms/step - loss: 84064.5391 - root_mean_squared_error: 289.9388 - val_loss: 84090.5078 - val_root_mean_squared_error: 289.9836
Epoch 3/100
767/767 ━━━━━━━━━━━━━━━━━━━━ 17s 22ms/step - loss: 84060.1328 - root_mean_squared_error: 289.9312 - val_loss: 84090.4922 - val_root_mean_squared_error: 289.9836
Epoch 4/100
767/767 ━━━━━━━━━━━━━━━━━━━━ 14s 18ms/step - loss: 84059.7578 - root_mean_squared_error: 289.9306 - val_loss: 84090.4844 - val_root_mean_squared_error: 289.9836
Epoch 5/100
767/767 ━━━━━━━━━━━━━━━━━━━━ 17s 22ms/step - loss: 84054.1953 - root_mean_squared_error: 289.9210 - val_loss: 84090.4688 - val_root_mean_squared_error: 289.9836
Epoch 6/100
767/767 ━━━━━━━━━━━━━━━━━━━━ 20s 26ms/step - loss: 84046.8750 - root_mean_squared_error: 289.9084 - val_loss: 84090.4688 - val_root_mea

In [12]:
lstm_dropout = Sequential()
lstm_dropout.add(Input(shape=(TIMESTEPS, N_FEATURES), 
                name='input'))
lstm_dropout.add(
    LSTM(units=16,
         activation='relu',
         return_sequences=True,
         recurrent_dropout=0.5,
         name='lstm_layer_1'))
lstm_dropout.add(Dropout(0.5))
lstm_dropout.add(
    LSTM(units=8,
         activation='relu',
         return_sequences=True,
         recurrent_dropout=0.5,
         name='lstm_layer_2'))
lstm_dropout.add(Flatten())
lstm_dropout.add(Dropout(0.5))
lstm_dropout.add(Dense(units=1,
                activation='linear', 
                name='output'))

lstm_dropout.compile(optimizer='adam',
              loss='mse',
              metrics=[
                  tf.keras.metrics.RootMeanSquaredError()
              ])
lstm_dropout_history = lstm_dropout.fit(x= np.asarray(X_train_scaled).astype('float32'),
                y=y_train,
                batch_size=128,
                epochs=100,
                validation_data=(np.asarray(X_valid_scaled).astype('float32'), 
                                y_valid),
                verbose=1).history

models.append(lstm_dropout)
rmse.append(lstm_dropout_history["val_root_mean_squared_error"][-1])

Epoch 1/100


2024-05-21 17:41:12.696929: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 86317440 exceeds 10% of free system memory.


767/767 ━━━━━━━━━━━━━━━━━━━━ 16s 12ms/step - loss: 41820.0078 - root_mean_squared_error: 199.3590 - val_loss: 873.0820 - val_root_mean_squared_error: 29.5480
Epoch 2/100
767/767 ━━━━━━━━━━━━━━━━━━━━ 9s 12ms/step - loss: 5418.0464 - root_mean_squared_error: 73.5250 - val_loss: 73.2977 - val_root_mean_squared_error: 8.5614
Epoch 3/100
767/767 ━━━━━━━━━━━━━━━━━━━━ 9s 12ms/step - loss: 3490.8257 - root_mean_squared_error: 59.0743 - val_loss: 30.4644 - val_root_mean_squared_error: 5.5195
Epoch 4/100
767/767 ━━━━━━━━━━━━━━━━━━━━ 8s 11ms/step - loss: 2907.2227 - root_mean_squared_error: 53.9158 - val_loss: 42.1942 - val_root_mean_squared_error: 6.4957
Epoch 5/100
767/767 ━━━━━━━━━━━━━━━━━━━━ 9s 11ms/step - loss: 2601.9661 - root_mean_squared_error: 51.0083 - val_loss: 17.4442 - val_root_mean_squared_error: 4.1766
Epoch 6/100
767/767 ━━━━━━━━━━━━━━━━━━━━ 8s 10ms/step - loss: 2440.3740 - root_mean_squared_error: 49.3995 - val_loss: 10.7929 - val_root_mean_squared_error: 3.2853
Epoch 7/100
767/7

In [ ]:
lstm_bidirectional = Sequential()
lstm_bidirectional.add(Input(shape=(TIMESTEPS, N_FEATURES), 
                name='input'))
lstm_bidirectional.add(
    LSTM(units=16,
         activation='tanh',
         return_sequences=True,
         recurrent_dropout=0.5,
         name='lstm_layer_1'))
lstm_bidirectional.add(Dropout(0.5))
lstm_bidirectional.add(
    LSTM(units=8,
         activation='tanh',
         return_sequences=True,
         recurrent_dropout=0.5,
         name='lstm_layer_2'))
lstm_bidirectional.add(Flatten())
lstm_bidirectional.add(Dropout(0.5))
lstm_bidirectional.add(Dense(units=1,
                activation='linear', 
                name='output'))
lstm_bidirectional.compile(optimizer='adam',
              loss='mse',
              metrics=[
                  tf.keras.metrics.RootMeanSquaredError()
              ])
lstm_bidirectional_history = lstm_bidirectional.fit(x= np.asarray(X_train_scaled).astype('float32'),
                y=y_train,
                batch_size=128,
                epochs=100,
                validation_data=(np.asarray(X_valid_scaled).astype('float32'), 
                                y_valid),
                verbose=1)

models.append(lstm_bidirectional)
rmse.append(lstm_bidirectional_history["val_root_mean_squared_error"][-1])

In [ ]:
best_model = models[np.argmin(rmse)]
p = best_model.predict(np.asarray(X_test_scaled).astype('float32'))
#res = pd.DataFrame(p)
#filename = "lstm_output_" + TARGET + ".csv"
#res.to_csv(filename, index=False)